In [1]:
import System.Random
import Control.Applicative (liftA3)
import Control.Monad (replicateM)
import Control.Monad.Trans.State
import qualified Data.DList as DL

In [2]:
data Die =
      DieOne
    | DieTwo
    | DieThree
    | DieFour
    | DieFive
    | DieSix
    deriving (Eq, Show)

In [3]:
intToDie :: Int -> Die
intToDie n =
    case n of
        1 -> DieOne
        2 -> DieTwo
        3 -> DieThree
        4 -> DieFour
        5 -> DieFive
        6 -> DieSix
        x -> error $ "intToDie got non 1-6 integer: " ++ show x

In [4]:
rollDieThreeTimes :: (Die, Die, Die)
rollDieThreeTimes = do
    let s = mkStdGen 0
        (d1, s1) = randomR (1, 6) s
        (d2, s2) = randomR (1, 6) s1
        (d3, _) =  randomR (1, 6) s2
    (intToDie d1, intToDie d2, intToDie d3)

In [5]:
roller = do
    (n, s) <- randomR (1, 6)
    return (intToDie n, s)

rollDie :: State StdGen Die
rollDie = state roller

In [6]:
rollDie' :: State StdGen Die
rollDie' = intToDie <$> state (randomR (1, 6))

rollDieThreeTimes' :: State StdGen (Die, Die, Die)
rollDieThreeTimes' = liftA3 (,,) rollDie rollDie rollDie

In [ ]:
evalState rollDieThreeTimes' (mkStdGen 9999)

In [ ]:
infiniteDie :: State StdGen [Die]
infiniteDie = repeat <$> rollDie

In [ ]:
take 6 $ evalState infiniteDie (mkStdGen 0)

In [ ]:
nDie :: Int -> State StdGen [Die]
nDie n = replicateM n rollDie

In [ ]:
evalState (nDie 5) (mkStdGen 0)

In [ ]:
rollsToGetTwenty :: StdGen -> Int
rollsToGetTwenty g = go 0 0 g
    where
        go :: Int -> Int -> StdGen -> Int
        go sum count gen
            | sum >= 20 = count
            | otherwise =
                let (die, nextGen) = randomR (1, 6) gen
                in go (sum + die) (count + 1) nextGen

In [ ]:
rollsToGetTwenty (mkStdGen 111)

In [ ]:
rollsToGetTwenty . mkStdGen <$> randomIO

In [ ]:
rollsToGetN :: Int -> StdGen -> Int
rollsToGetN n g = go 0 0 g
    where
        go :: Int -> Int -> StdGen -> Int
        go sum count gen
            | sum >= n = count
            | otherwise =
                let (die, nextGen) = randomR (1, 6) gen
                in go (sum + die) (count + 1) nextGen

In [ ]:
rollsToGetN 100 . mkStdGen <$> randomIO

In [ ]:
rollsCountLogged :: Int -> StdGen -> (Int, [Die])
rollsCountLogged n g = go 0 (0, []) g
    where
        go :: Int -> (Int, [Die]) -> StdGen -> (Int, [Die])
        go sum (count, history) gen
            | sum >= n = (count, history)
            | otherwise =
                let (die, nextGen) = randomR (1, 6) gen
                in go (sum + die) (count+1, history ++ [intToDie die]) nextGen

In [ ]:
rollsCountLogged 100 . mkStdGen <$> randomIO

<h4>Custom State</h4>

In [7]:
newtype Moi s a = Moi { runMoi :: s -> (a, s) }

In [8]:
{-# LANGUAGE InstanceSigs #-}

instance Functor (Moi s) where
    fmap :: (a -> b) -> Moi s a -> Moi s b
    fmap f (Moi g) = Moi $ \s -> let (x, s') = g s in (f x, s')

In [9]:
runMoi ((+1) <$> (Moi $ \s -> (0, s))) 0

Line 1: Use tuple-section
Found:
\ s -> (0, s)
Why not:
(0,)

(1,0)

In [20]:
instance Applicative (Moi s) where
    pure :: a -> Moi s a
    pure a = Moi $ \s -> (a, s)
    
    (<*>) :: Moi s (a -> b) -> Moi s a -> Moi s b
    (<*>) (Moi f) (Moi g) = Moi $ \s -> let (x, s') = g s
                                            (h, s'') = f s'
                                        in  (h x, s'')
    
{---
g ~ s -> (a, s)
f ~ s -> (a -> b, s)
res ~ s -> (b, s)
---}

Line 3: Use tuple-section
Found:
\ s -> (a, s)
Why not:
(a,)

In [11]:
x = Moi $ (,) (+1)
y = x <*> (Moi $ \s -> (0, s))

runMoi y 2500

(1,2500)

In [22]:
instance Monad (Moi s) where
    return = pure
    
    (>>=) :: Moi s a -> (a -> Moi s b) -> Moi s b
    (Moi f) >>= g = Moi $ \s -> let (x, s') = f s in runMoi (g x) s'
    
{--
f ~ s -> (a, s)
g ~ a -> s -> (b, s)
res ~ s -> (b, s)
--}

In [13]:
get :: Moi s s
get = Moi $ \s -> (s, s)

runMoi get "curryIsAmazing"

("curryIsAmazing","curryIsAmazing")

In [14]:
put :: s -> Moi s ()
put a = Moi $ \s -> ((), a)

runMoi (put "blah") "woot"

((),"blah")

In [15]:
exec :: Moi s a -> s -> s
exec msa s = snd $ (runMoi msa) s

exec (put "wilma") "daphne"
exec get "scooby doo"

"wilma"

"scooby doo"

In [16]:
eval :: Moi s a -> s -> a
eval msa = fst . (runMoi msa)

eval get "bunny"

"bunny"

In [23]:
modify :: (s -> s) -> Moi s ()
modify f = Moi $ \s -> ((), f s)

runMoi (modify (+1) >> modify (+1)) 0

((),2)

<h4>FizzBuzz</h4>

In [ ]:
fizzBuzz :: Integer -> String
fizzBuzz n | n `mod` 15 == 0 = "FizzBuzz"
           | n `mod` 5 == 0 = "Buzz"
           | n `mod` 3 == 0 = "Fizz"
           | otherwise = show n

In [ ]:
addResult :: Integer -> State [String] ()
addResult n = do
    xs <- get
    let result = fizzBuzz n
    put (result : xs)

fizzBuzzList :: [Integer] -> [String]
fizzBuzzList list = execState (mapM_ addResult list) []

In [ ]:
mapM_ putStrLn $ reverse $ fizzBuzzList [1..100]

In [ ]:
addResult' :: Integer -> State (DL.DList String) ()
addResult' n = do
    xs <- get
    let result = fizzBuzz n
    put (DL.snoc xs result)

fizzBuzzList' :: [Integer] -> [String]
fizzBuzzList' list = let dlist = execState (mapM_ addResult' list) DL.empty
                     in DL.apply dlist []

In [ ]:
mapM_ putStrLn $ fizzBuzzList' [1..100]